# Topic modeling 

In this notebook I will explore topic modeling for the 20newsgroups dataset. 
The notebook flow is as follows:
* Import the dataset and initial preprocessing/cleaning
* Use the spacy package to tokenize, lemmatize and create a tri-gram model for the documents
* Perform topic modeling using Latent Dirichlet Allocation (LDA) and Non-negative Matrix Factorization (NMF)
* Measure the 'goodness' of the LDA and NMF models. 
* Apply the the resulting model to unseen documents to derive its topic.

Some spacy installation notes: 
* In order to install spacy visual studio build tools is needed http://landinghub.visualstudio.com/visual-cpp-build-tools
* To download the spacy english model in python 3 execute this command as admin: python -m spacy download en

Import the necessary packages and functions.

In [1]:
import spacy
import pandas as pd
import itertools as it
import os
import pickle 
from sklearn.datasets import fetch_20newsgroups
from numpy import exp
import string
import numpy as np
from math import sqrt,isnan
from gensim.models import Phrases

C:\Users\venciso\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Get the documents and perform some initial cleaning, i.e., there are newline and tab characters spread across the documents which become a problem later on if not removed.
Make everything lower-case.

In [2]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
dataset.data = [d.replace('\n', ' ') for d in dataset.data]
dataset.data = [d.replace('\t', ' ') for d in dataset.data]
dataset.data = [d.replace("'","") for d in dataset.data]
dataset.data = [d.lower() for d in dataset.data]

Get the number of documents in the dataset

In [3]:
len(dataset.data)

11314

After some initial modeling of the data it became clear that there were a few documents that were just gibberish and were affecting the results. 

Onse such example can be seen below.

In [4]:
bad_docs=[doc for doc in dataset.data if doc.find('ax>ax>') != -1]
bad_docs[0]

'  ------------ part 11 of 14 ------------ mr1865%22dm75u=75u4)"0iv=g9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v= mg9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v= mg9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v=g9v= mg9v=0m75u=625!;)b8e,3`]g9s2+[>wm4qd]f0->7exjwz5f,8>ax>ax> max>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax> max>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax> max>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax> max>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax>ax> max>ax<q9j5znp.9f3v9f9f9f3w2tg%q&1fpl%-3[>wm[>wm[5-3l+!3l+`9 m&1d9&7%q<7%q<7%q&72tct]/=+2tm+2tm+2/3t]/3t]/=+2tg%qtcv9f0,# m`p->7ez[n[lj>gizw]]1z6ei0l+9f9f9f9f9f;$q,3$q,3$q,3$9f8+"pl+ mi:5>ghj*bhjn[m>7ey>7@.9/3w2tg%q<1d9l%-3[8n+-,5g9p],3$q,3$q, m3$r)b8f)b8g)r<g)r<g)r<g)r186r<g)r<g)r<g)r<g)r<f)b8f)b8f)b8f) mb8g)r<g)r<g)r<f)b8e,3$q,3$q,3$q,3(f)b8f)b8f)b8f)b4q,3&?%-(om m[5-34u.p&72/3v9f0,#`uy>7@,#`yf9f3t]/3t]f9d]f9f9f3t]/3w2tm+2 m<7$9&1fpl+!3l!d9&1d9&7%q&1d9l+`9&;"p&1d9<=(]/9f9f9d#`uz[nriz mw]_?w]_?w]_?>giz>giz*bhj>giz>bhj

I decided to remove these kind of documents. 

Removing the patterns in the following lines get rid of most of the issues. 

In [5]:
dataset.data=[doc for doc in dataset.data if doc.find('ax>ax>') == -1]
dataset.data=[doc for doc in dataset.data if doc.find('=_') == -1]
len(dataset.data)

11299

I will also remove most punctuation from the documents. 

I am only keeping commas and full stops.

In [6]:
myPunct='!"#$%&\'()*+-/:;<=>?@[\\]^_`{|}~'

documents=[]
for doc in dataset.data:
    documents.append(" ".join("".join([ch for ch in doc if ch not in myPunct]).split()))

Take a document at random and see how it looks before and after.

In [7]:
dataset.data[124]

'  well, there is a fair amount of evidence floating around that indicates that oto has been around since at least the late 1800s, long before crowley ever heard of it, how long has amorc been around? (yes, i know that they claim to have existed as an organization clear into prehistory, but i doubt that they have any organizational paperwork as a non-profit that can be carbon-dated to 20,000 bc)                                              a.lizard '

In [8]:
documents[124]

'well, there is a fair amount of evidence floating around that indicates that oto has been around since at least the late 1800s, long before crowley ever heard of it, how long has amorc been around yes, i know that they claim to have existed as an organization clear into prehistory, but i doubt that they have any organizational paperwork as a nonprofit that can be carbondated to 20,000 bc a.lizard'

The documents can saved for future use. 

In [9]:
save_documents = open("documents_stripped.pickle", "wb")
pickle.dump(dataset.data, save_documents)
save_documents.close()
# open_file = open("documents_stripped.pickle", "rb")
# documents = pickle.load(open_file)
# open_file.close()

### **Document parsing**

Load the spacy english package.

In [10]:
nlp = spacy.load('en')

Spacy is very convenient to use as it does a lot of the heavy lifting with just one call.

For example, calling 'nlp(text)' will split the text into sentences, apply part of speech tagging, lemmatization, entity type analysis, punctuation, whitespace and stopword detection and more. 

Here I create a fucntion, lemmatized_sentence_corpus, that yields lemmatized sentences with no punctuation, whitespaces or stopwords. 

One thing to note is that I'm removing stopwords twice. First via the punct_space function and later in the function I check the lemmatized tokens again. The reason I do this is that spacy will take words like 'wont' and 'dont' and will lemmatize them to 'will not' and 'do not'. The second pass of stopword removal gets rid of these cases.

In [11]:
def punct_space(token):
    return token.is_punct or token.is_space or token.is_stop

def lemmatized_sentence_corpus(docs):
    for parsed_doc in nlp.pipe(docs):
        
        for sent in parsed_doc.sents:
            token_list=[]
            for token in sent:
                if not punct_space(token):
                    if token.lemma_ == '-PRON-':   #There seems to be a bug in the lemmatizing that assigns '-PRON-' to pronouns 
                                                   #I will handle the exception here and raise with the package owner 
                        token_list.append(token.orth_)
                    elif token.lemma_ not in spacy.en.STOP_WORDS:
                        token_list.append(token.lemma_)
            
            yield ' '.join(token_list)

#Note: spacy takes things like wont and dont and splits them in to will not/do not. Do another stopword removal step here, hopefully that 
#takes care of the problem. 

Apply the 'lemmatized_sentence_corpus' function to the documents and get back a set of sentences.  

In [12]:
%%time
unigram_sentences=[]
for sentence in lemmatized_sentence_corpus(documents):
    unigram_sentences.append(sentence)

Wall time: 1min 38s


Check a few of the resulting sentences. 

In [13]:
for sent in unigram_sentences[30240:30250]:
    print(sent)
    print(u'')

sell bike net young lady live salt lake city

live near lose angele

turn mutual aquaintance ucla

uh right

forget rbi bar hr base

fraction run come solo hr run score happen player base batter good

use phrase happen advisedly

lot people try figure player ability turn notch clutchrbiwhatever situation find evidence ability measurable extent

clutch hitter

people tend thing cause rbi somebody base end rbis proportional teammate oblige position agree



Now that we have the tokenized sentences we can look for pairs of words that appear next to each other more than it would be expected by chance. This words can be joined together to create a bi-gram, i.e., combine them to make a single token that will be more informative to the topic modelling algorithm 

An example of this could be the words 'ice' and 'cream'. They might be combiined to create an 'ice_cream' token which will provide more information that the single tokens would. 

The bi-gram transofrmation is performed using the gensim Phrases function.

In [14]:
unigram_sentences = [sent.split(" ") for sent in unigram_sentences]

In [15]:
bigram_model = Phrases(unigram_sentences)

In [16]:
%%time
bigram_sentences=[]
for sentence in unigram_sentences:
    bigram_sentences.append(' '.join(bigram_model[sentence]))

C:\Users\venciso\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\gensim\models\phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Wall time: 4.97 s


Let's examine some of the sentences again now that the bi-grams have been created. 

The model identified 'salt' and 'lake' as appearing together very frequently and joined them into 'salt_lake'.

In [17]:
for sent in bigram_sentences[30240:30250]:
    print(sent)
    print(u'')

sell bike net young lady live salt_lake city

live near lose angele

turn mutual aquaintance ucla

uh right

forget rbi bar hr base

fraction run come solo hr run_score happen player base batter good

use phrase happen advisedly

lot people try_figure player ability turn notch clutchrbiwhatever situation find evidence ability measurable extent

clutch hitter

people tend thing cause rbi somebody base end rbis proportional teammate oblige position agree



Let's do one more pass and create tri-grams. 

In [18]:
bigram_sentences = [sent.split(" ") for sent in bigram_sentences]

In [19]:
trigram_model = Phrases(bigram_sentences)

In [20]:
%%time
trigram_sentences=[]
for sentence in bigram_sentences:
    trigram_sentences.append(' '.join(trigram_model[sentence]))

C:\Users\venciso\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\gensim\models\phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Wall time: 4.16 s


After this pass the model now adds 'city' into 'salt_lake' to form a tri-gram. 

In [21]:
for sent in trigram_sentences[30240:30250]:
    print(sent)
    print(u'')

sell bike net young lady live salt_lake_city

live near lose angele

turn mutual aquaintance ucla

uh right

forget rbi bar hr base

fraction run come solo hr run_score happen player base batter good

use phrase happen advisedly

lot_people try_figure player ability turn notch clutchrbiwhatever situation find evidence ability measurable extent

clutch hitter

people tend thing cause rbi somebody base end rbis proportional teammate oblige position agree



Now that we have finished processing the sentences we can apply the process to the documents using the 'transform_documents' function. 

In [22]:
def transform_documents(docs):
    trigram_transformed_docs = []
    for parsed_doc in nlp.pipe(docs):
        unigram_doc=[]


        for token in parsed_doc:
            if not punct_space(token):
                if token.lemma_ == '-PRON-':
                    unigram_doc.append(token.orth_)
                elif token.lemma_ not in spacy.en.STOP_WORDS:
                    unigram_doc.append(token.lemma_)

        bigram_doc = bigram_model[unigram_doc]
        trigram_doc = trigram_model[bigram_doc]
        trigram_doc = [term for term in trigram_doc]

        trigram_doc = ' '.join(trigram_doc)
        trigram_doc = ' '.join( [term for term in trigram_doc.split() if len(term)>1] )

        trigram_transformed_docs.append(trigram_doc)
    return trigram_transformed_docs

In [23]:
trigram_transformed_docs = transform_documents(documents)

C:\Users\venciso\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\gensim\models\phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


We can examine a document before and after the transformation. 

In [24]:
i=0
print('Original:' + '\n')

print(documents[i])

print ('----' + '\n')
print ('Transformed:' + '\n')

print(trigram_transformed_docs[i])

Original:

well im not sure about the story nad it did seem biased. what i disagree with is your statement that the u.s. media is out to ruin israels reputation. that is rediculous. the u.s. media is the most proisraeli media in the world. having lived in europe i realize that incidences such as the one described in the letter have occured. the u.s. media as a whole seem to try to ignore them. the u.s. is subsidizing israels existance and the europeans are not at least not to the same degree. so i think that might be a reason they report more clearly on the atrocities. what is a shame is that in austria, daily reports of the inhuman acts commited by israeli soldiers and the blessing received from the government makes some of the holocaust guilt go away. after all, look how the jews are treating other races when they got power. it is unfortunate.
----

Transformed:

sure story nad biased disagree statement u.s media ruin israel reputation rediculous u.s media proisraeli medium world liv

### **Topic modelling**

We are now ready to take the tokenized tri-gram documents and perform topic modelling.

For LDA we will use the gensim package and for NMF we will use scikit-learn.

Let's say we have a collection of N documents, LDA assumes that each document is generated by a mixture of topics and that the number of topics is usually small. Also, each topic in turn generates a small set of words. 
LDA assumes that documents are generated by choosing words according to a multinomial distributions (one for topics and one for words in the topics) whose hypeparamters are governed by sparse Dirichlet priors (this basically encodes the 'small' concept in the previous paragraph).
LDA the tries to backtrack from this process to find a set of topics that are likely to have generated the documents. 

NMF represents the set of documents as a matrix X of size N (documents) columns and M (words) rows. It then tries to find two non-negative matrices W and H such that their product approximates X as well as possible. W is an N by K and H is K by N so it can be seen that K is the number of topics.

LDA has a frequentist version called probabilistic latent semantic analysis (pLSA). NMF and pLSA are equivalent and the main difference between them and LDA is the sparsity introduced by the Dirichlet priors.
LDA has more control over the way the topics are specified since one can modify the prior hyperparameters to achieve more or less distinct topics. 


In both LDA and NMF techniques the user has to specify the number of topics in the model. While researching online there does not seem to be a definitive way to determine the number of topics. Using the perplexitiy statistic or some sort of elbow method similar to what is used in cluster analysis are some of the ideas I found but there doesn't seem to be a general consensus on these. 

For this reason and given that we know the data is generated from 20 newsgroups I will choose 20 as the obvious choice for the number of topics.

In [25]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models import LdaModel,TfidfModel
from gensim.matutils import cossim
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer 

For LDA modelling we need to create a dictionary of all the tokens. 

We then filter out the tokens that are either very common or very rare or we can decide to keep a certan number of tokens. 

From the dictionary we can create a bag of words which have all the tokens that we kept and how many times they occur in each document. 

The resulting bag of words corpus can be fed into the lda model. 

In [26]:
trigram_docs = [doc.split(" ") for doc in trigram_transformed_docs]

trigram_dictionary=Dictionary(trigram_docs)

trigram_dictionary.filter_extremes(no_above=0.5,no_below=10,keep_n=5000)
trigram_dictionary.compactify()

trigram_bow_corpus=[trigram_dictionary.doc2bow(doc) for doc in trigram_docs]

lda=LdaModel(trigram_bow_corpus,num_topics=20,id2word=trigram_dictionary)

The alpha parameter controls the sparsity of the document to topic priors. It is usually set to a deafult value of 1 / number_of_topics.

Run a version of the model with an alpha > 1 to explore the behaviour. 

In [27]:
lda_a=LdaModel(trigram_bow_corpus,num_topics=20,id2word=trigram_dictionary,alpha=3)

In order to perform NMF we need a term-frequency inverse-document-frequency representation. 

We first use TfidfVectorizer to decide the features in the model. The nuber of features we will use will be the same as for LDA. 

We then transform the documents and feed them into the NMF function. 

In [28]:
trigram_transformed_docs_str=[str(text) for text in trigram_transformed_docs]
tfidf_vectorizer = TfidfVectorizer(min_df=10, max_df=0.8,max_features=5000)
tfidf = tfidf_vectorizer.fit_transform(trigram_transformed_docs_str)

nmf = NMF(n_components=20,init='nndsvd').fit(tfidf)

In [29]:
len(tfidf_vectorizer.get_feature_names())

5000

We define functions to explore the topics generated by each algorithm. 

For each of the topics we show the top n most important terms. 

In [30]:
def get_lda_topics(model,num_topics,n_top_words):
    word_dict = {};
    for i in range(num_topics):
        words = eval("{}.show_topic(i, topn = n_top_words)".format(model))
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

def get_nmf_topics(model,num_topics,n_top_words):
    feat_names = tfidf_vectorizer.get_feature_names()
    word_dict = {};
    for i in range(num_topics):
        words_ids = eval("{}.components_[i].argsort()[:-n_top_words - 1:-1]".format(model))
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

**LDA topics**

In [31]:
get_lda_topics('lda',20,10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20
0,key,entry,think,gun,think,law,day,armenian,game,program,thank,good,program,god,think,10,use,jew,drive,know
1,use,,good,use,know,pain,power,people,team,available,post,car,work,people,problem,12,window,jewish,pin,people
2,system,file,know,people,people,patient,think,president,year,include,memory,time,line,know,time,11,system,israel,jumper,like
3,chip,program,player,time,like,doctor,time,russian,good,version,address,like,file,christian,know,15,problem,state,gm,israel
4,new,driver,like,like,believe,disease,water,turkish,win,information,motherboard,look,know,jesus,good,13,file,claim,insert,want
5,car,function,people,work,exist,tax,know,government,play,file,scsi,bike,use,think,science,14,know,israeli,italy,problem
6,price,work,case,good,want,state,space,think,season,server,system,right,like,believe,objective,16,work,question,doug,time
7,datum,set,point,know,way,know,easter,mr,player,use,list,come,year,come,true,25,run,post,slave,kill
8,phone,try,thing,want,find,year,work,city,hit,widget,include,think,post,thing,moral,20,display,force,setup,right
9,know,find,right,think,mean,medical,good,turkey,like,source,clone,way,group,word,like,18,software,article,17,tell


In [32]:
get_lda_topics('lda_a',20,10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20
0,know,know,think,like,use,god,people,like,know,file,know,know,know,think,good,people,know,game,think,good
1,like,good,people,know,run,christian,armenian,think,good,program,like,armenian,think,know,think,know,time,team,work,time
2,think,like,like,good,system,jesus,turkish,know,think,use,good,people,time,good,like,like,good,10,know,think
3,good,think,know,think,program,people,jew,people,like,include,use,come,good,like,know,good,use,win,time,work
4,people,people,good,use,window,think,know,good,time,available,work,tell,people,people,time,time,gun,12,people,use
5,work,problem,time,want,good,good,turkey,time,year,information,think,think,like,use,people,use,people,14,use,want
6,time,use,use,people,problem,believe,like,work,people,system,people,time,problem,problem,work,problem,file,13,like,like
7,way,want,work,try,line,thing,time,want,way,software,thing,like,want,want,try,way,year,11,good,know
8,try,work,want,time,include,way,good,use,work,image,want,want,thing,way,use,think,like,15,want,people
9,look,find,way,work,work,know,think,problem,want,version,time,kill,try,time,system,find,think,16,program,thing


We can see the effect of changing the alpha parameter by examining terms like 'think' or 'time'. These terms appear in the top words of a few of the topics of the 'lda' model but they appear in almost every topic in 'lda_a'.

The default parameter for alpha is 1/n_topics, more sparcity could be introduced if desired by making this smaller.

**NMF topics**

In [33]:
get_nmf_topics('nmf',20,10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20
0,people,window,game,time,god,thank,chastity_intellect_gebcadre,key,car,israel,think,drive,know,edu,file,card,good,com,bike,look
1,right,program,team,post,christian,hi,surrender_soon,chip,engine,armenian,like,scsi,want,00,directory,problem,bad,article,ride,email
2,government,run,player,work,jesus,email,edu_shameful,use,price,arab,guy,disk,like,sale,zip,driver,thing,list,motorcycle,information
3,gun,use,play,find,bible,information,dsl,system,dealer,israeli,mean,jumper,thank_advance,email,format,system,player,sun,mile,thank_advance
4,law,application,year,like,believe,info,gordon_bank_n3jxp_skepticism,government,new,jew,way,boot,hear,sell,convert,monitor,year,email,helmet,buy
5,state,display,win,group,faith,appreciate,pitt,encryption,mile,kill,wrong,format,sure,offer,disk,work,way,reply,sell,want
6,country,font,season,read,religion,interested,patient,nsa,model,jewish,yes,switch,thing,include,exe,apple,lot,internet,rid,find
7,mean,server,fan,new,sin,find,migraine,clipper,tire,palestinian,believe,cable,tell,send,ftp,mode,point,quote,honda,interested
8,want,screen,hockey,space,christ,need,probably,phone,ford,attack,lot,hard_drive,maybe,ftp,program,machine,use,address,buy,model
9,person,version,score,try,life,send,weight,algorithm,power,war,thing,ide,ne,mit,download,memory,buy,hp,turn,package


At first glance, NMF topics seem to be quite distinct. There does not seem to be a lot of term repetition per topic. 


### **Assessing model quality**

We have two different techniques and we would like to get a measure of how good they are. 

The 20newsgroups dataset has some extra test documents that we have not used in our modelling. We can somehow use these to assess our models. 

First we apply the same data cleaning, pre-processing and tri-gram modelling to the test documents: 

In [34]:
testDataset=fetch_20newsgroups(shuffle=True,random_state=1,remove=('headers', 'footers', 'quotes'),subset='test')

testDataset.data = [d.replace('\n', ' ') for d in testDataset.data]
testDataset.data = [d.replace('\t', ' ') for d in testDataset.data]
testDataset.data = [d.replace("'","") for d in testDataset.data]
testDataset.data = [d.lower() for d in testDataset.data]

testDataset.data=[doc for doc in testDataset.data if doc.find('ax>ax>') == -1]
testDataset.data=[doc for doc in testDataset.data if doc.find('=_') == -1]

test_documents=[]
for doc in testDataset.data:
    test_documents.append(" ".join("".join([ch for ch in doc if ch not in myPunct]).split()))

In [35]:
save_documents = open("test_documents_stripped.pickle", "wb")
pickle.dump(test_documents, save_documents)
save_documents.close()

open_file = open("test_documents_stripped.pickle", "rb")
test_documents = pickle.load(open_file)
open_file.close()

We will try to assess the quality of the documents by using cosine similarity.

We split each test document in two halves and we apply the LDA/NMF model to each of the halves. 

If the model is good then the topics arising from halves of the same document should be the same most of the time. Similarly, topics arising from halves from different documents should be different in general. 

We can assess how similar or different the topics are by computing the cosine similarity of the resulting vectors.

Similarities closer to 1 mean that the vectors have the same direction. Similarities closer to 0 mean that the vectors differ in direction. 

In [36]:
def cosine_sim(a, b):
    if len(a) != len(b):
        raise(ValueError, "a and b must be same length")
    numerator = 0
    denoma = 0
    denomb = 0
    for i in range(len(a)):       
        ai = a[i]             
        bi = b[i]
        numerator += ai*bi    
        denoma += ai*ai       
        denomb += bi*bi
    result = numerator / (sqrt(denoma)*sqrt(denomb))
    return result

def lda_model(tokenized_docs,keep_num,alpha_val,n_topics=20):
    trigram_dict=Dictionary(tokenized_docs)
    trigram_dict.filter_extremes(keep_n=keep_num)
    trigram_dict.compactify()
    print("Number of features in model: {}".format(len(trigram_dict)))
    
    trigram_bow_corp = [trigram_dict.doc2bow(doc) for doc in tokenized_docs]

    ldamod=LdaModel(trigram_bow_corp,num_topics=n_topics,id2word=trigram_dict,alpha=alpha_val)
    
    return (ldamod,trigram_dict)

def nmf_model(tokenized_docs,keep_num,n_topics=20):
    tokenized_docs_str=[str(text) for text in tokenized_docs]
    tfidf_vectorizer = TfidfVectorizer(max_features=keep_num)
    tfidf = tfidf_vectorizer.fit_transform(tokenized_docs_str)

    nmf = NMF(n_components=20,init='nndsvd').fit(tfidf)
    return(nmf,tfidf_vectorizer)

def intra_inter(model_type,model_name,dict_or_vect,test_docs, num_pairs=10000):
    if model_type == 'lda':
        part1 = eval("{}[[{}.doc2bow(tokens.split()[:round(len(tokens.split())/2)]) for tokens in test_docs]]".format(model_name,dict_or_vect))
        part2 = eval("{}[[{}.doc2bow(tokens.split()[round(len(tokens.split())/2):]) for tokens in test_docs]]".format(model_name,dict_or_vect))
        
        intra_mean = np.mean([cossim(p1, p2) for p1, p2 in zip(part1, part2)])
        print("average cosine similarity between corresponding parts (higher is better): {}".format(intra_mean))

        random_pairs = np.random.randint(0, len(test_docs), size=(num_pairs, 2))
        inter_mean=np.mean([cossim(part1[i[0]], part2[i[1]]) for i in random_pairs])
        print("average cosine similarity between {} random parts (lower is better): {}".format(num_pairs,inter_mean))    
        
    elif model_type == 'nmf':
        part1=eval("[{}.transform({}.transform(str(text.split()[:round(len(text.split())/2)]) for text in test_docs))]".format(model_name,dict_or_vect))
        part2=eval("[{}.transform({}.transform(str(text.split()[round(len(text.split())/2):]) for text in test_docs))]".format(model_name,dict_or_vect))
        
        cos = [cosine_sim(p1, p2) for p1, p2 in zip(part1[0], part2[0])]
        intra_mean = np.mean([0 if isnan(c) else c for c in cos])
        print("average cosine similarity between corresponding parts (higher is better): {}".format(intra_mean))

        random_pairs = np.random.randint(0, len(test_docs), size=(num_pairs, 2))
        cos =[cosine_sim(part1[0][i[0]],part2[0][i[1]]) for i in random_pairs]
        inter_mean=np.mean([0 if isnan(c) else c for c in cos])
        print("average cosine similarity between {} random parts (lower is better): {}".format(num_pairs,inter_mean))    
        
    else:
        print('Error: only lda or nmf models are supported')
    


After initial exploration of the results I was getting quite low 'intra' cosine similarities. I attribute this to the fact the that for shorter documents the two halves might be inherently more different. In order to be fair to the model I only keep documents to test that have more than 150 words. 

I will use 1000 such documents

In [37]:
test_doc_text=[text for text in test_documents if len(text.split())>150]
test_doc_text=test_doc_text[:1000]
test_doc_trans=transform_documents(test_doc_text)

C:\Users\venciso\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\gensim\models\phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [38]:
intra_inter('lda','lda','trigram_dictionary',test_doc_trans,num_pairs=1000)

average cosine similarity between corresponding parts (higher is better): 0.6923462505094791
average cosine similarity between 1000 random parts (lower is better): 0.18954029516243434


In [39]:
intra_inter('lda','lda_a','trigram_dictionary',test_doc_trans,num_pairs=1000)

average cosine similarity between corresponding parts (higher is better): 0.9789740115559439
average cosine similarity between 1000 random parts (lower is better): 0.8273055423856238


In [40]:
intra_inter('nmf','nmf','tfidf_vectorizer',test_doc_trans,num_pairs=1000)

C:\Users\venciso\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\ipykernel\__main__.py:13: RuntimeWarning: invalid value encountered in double_scalars


average cosine similarity between corresponding parts (higher is better): 0.6468048359264356
average cosine similarity between 1000 random parts (lower is better): 0.24065869789743066


Examining the initial models it seems that LDA slightly outperforms NMF. 

The 'lda' model with default paramters seems to have a good balance of inter and intra cosine similarity metrics. 

The 'lda_a' model has really good intra similarity but the inter similarity becomes really high too. This makes sense since topics are more interrelated to each other.

The 'nmf' performs similarly to 'lda' but it has a lower intra similarity and a higher inter similarity.

We can play with some of the parameters of the model to see if we can arrive to a concensus model we can use to predict new docuemnts. 

The two main parameters I will change are the alpha parameter in LDA and the number of features.

In [41]:
for a in [1/100,1/20]:
    for k in [1000,2500,5000,10000]:
        print("Alpha: {}, Number of features: {}".format(a,k))
        model=lda_model(trigram_docs,keep_num=k,n_topics=20,alpha_val=a)
        intra_inter(model_type='lda',model_name='model[0]',dict_or_vect='model[1]',test_docs=test_doc_trans,num_pairs=1000)      
        print("")

Alpha: 0.01, Number of features: 1000
Number of features in model: 1000
average cosine similarity between corresponding parts (higher is better): 0.5550080791214435
average cosine similarity between 1000 random parts (lower is better): 0.14612617207136527

Alpha: 0.01, Number of features: 2500
Number of features in model: 2500
average cosine similarity between corresponding parts (higher is better): 0.6435816465555664
average cosine similarity between 1000 random parts (lower is better): 0.15488015936019536

Alpha: 0.01, Number of features: 5000
Number of features in model: 5000
average cosine similarity between corresponding parts (higher is better): 0.6740827657376589
average cosine similarity between 1000 random parts (lower is better): 0.17079694628881598

Alpha: 0.01, Number of features: 10000
Number of features in model: 10000
average cosine similarity between corresponding parts (higher is better): 0.7070973265973708
average cosine similarity between 1000 random parts (lower is 

In [ ]:
for k in [1000,2500,5000,10000]:
    print("Number of features: {}".format(k))
    model=nmf_model(trigram_docs,keep_num=k,n_topics=20)
    intra_inter(model_type='nmf',model_name='model[0]',dict_or_vect='model[1]',test_docs=test_doc_trans,num_pairs=1000)      
    print("")

An LDA model with alpha= 1/100 and number of features = 5000 has a good balance of intra and inter cosine similarity metrics. 

Let's train a model with these parameters and keep it for prediction purposes. 

In [52]:
ldaModel = lda_model(trigram_docs,keep_num=5000,n_topics=20,alpha_val=0.01)

Number of features in model: 5000


In [53]:
get_lda_topics('ldaModel[0]',20,20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19,Topic # 20
0,think,people,report,god,game,use,key,gun,know,drive,armenian,people,good,people,bike,program,know,program,window,think
1,like,good,good,christian,team,file,message,file,people,know,turkey,government,car,state,israel,use,come,satellite,file,people
2,good,disease,time,book,win,work,system,church,use,system,russian,right,work,fbi,car,line,people,include,use,good
3,year,word,year,greek,play,system,de,people,key,problem,turk,law,use,right,like,run,think,space,display,know
4,know,claim,age,bible,player,software,use,find,way,like,german,gun,like,want,think,problem,tell,nasa,widget,thing
5,time,time,increase,belief,year,include,information,study,good,apple,jew,country,think,government,arab,window,god,information,application,believe
6,come,patient,know,science,10,run,list,point,like,computer,patent,state,power,true,good,error,like,mission,server,point
7,day,god,number,good,season,available,rsa,area,time,use,turkish,think,time,israel,time,file,jesus,1993,support,way
8,people,world,1992,religion,12,version,work,problem,think,work,jewish,armenian,new,mean,look,,die,project,datum,mean
9,thing,think,stat,vs,good,like,security,way,want,font,p.,try,problem,way,israeli,write,want,research,include,reason


Let's examine the topics and try to assign names to them. Going through the 20 topics is interesting, we can see some of them are not immediately straightforward to categorise and others can be interpreted as a mix of two different topics but in general one can identify a thread through the topic words. 
Here we can also see that some funny terms are still showing up even after we made efforts to clean the documents. The claening process can therefore probably be further optimised. 
One other note is that even though the number of topics is supposed to be 20 there are a few instances where topics can be the repeated across groups. 

In [54]:
topic_names={
    0: 'miscellaneous',
    1: 'medicine',
    2: 'miscellaneous 1',
    3: 'religion',
    4: 'sports',
    5: 'software',
    6: 'encryption',
    7: 'guns',
    8: 'politics',
    9: 'computing',
    10: 'foreign politics', 
    11: 'politics',
    12: 'cars',
    13: 'politics 1',
    14: 'middle east/cars mix',
    15: 'computing',
    16: 'christianity',
    17: 'space',
    18: 'computing 1',
    19: 'religious/atheist discussion'
}

### **Prediction of unseen documents**

The dataset I chose to use actually has the news gorup labels for each document. This means that I could have predicted the labels with both LDA and NMF models and assessed which one was better based on the accuracy.

I decided not to pursue this for a couple of reasons:
* We know the 'answers' for this particular dataset but in general the labels will be unknown so it will not be possible to apply this kind of method.
* The fact that a comment was left in a particular newsgroup doesn't necessarily mean that it is about the same topic. It will be the case in general but nobody prevent someone going into the 'atheism' group and talk about something completely different. My point being that the labels are not a perfect representation of 'topics'

For curiosity's sake, below are the real 'topics'. If we examine the names we can see that indeed a few topics are interrelated/repeated. The topics that we assigned don't match exactly one to one to these topics but there is good coverage of the different themes. 

In [55]:
real_topic_names={}
for int,cat in enumerate(testDataset.target_names):
    real_topic_names[int]=cat
real_topic_names

{0: 'alt.atheism',
 1: 'comp.graphics',
 2: 'comp.os.ms-windows.misc',
 3: 'comp.sys.ibm.pc.hardware',
 4: 'comp.sys.mac.hardware',
 5: 'comp.windows.x',
 6: 'misc.forsale',
 7: 'rec.autos',
 8: 'rec.motorcycles',
 9: 'rec.sport.baseball',
 10: 'rec.sport.hockey',
 11: 'sci.crypt',
 12: 'sci.electronics',
 13: 'sci.med',
 14: 'sci.space',
 15: 'soc.religion.christian',
 16: 'talk.politics.guns',
 17: 'talk.politics.mideast',
 18: 'talk.politics.misc',
 19: 'talk.religion.misc'}

We can apply our LDA model to a particular unseen document and get a probability of the document having been generated by a number of topics. 

In order do this we create another function that processes the document, applies the LDA model and spits out a prediction based on the topics that have the highest probability og having generated that document. 

In [56]:
def takeSecond(elem):
    return elem[1]

def lda_description(text, min_topic_freq=0.05):
    parsed_doc = nlp(text)
    
    unigram_doc=[]

    for token in parsed_doc:
        if not punct_space(token):
            if token.lemma_ == '-PRON-':
                unigram_doc.append(token.orth_)
            elif token.lemma_ not in spacy.en.STOP_WORDS:
                unigram_doc.append(token.lemma_)

    bigram_doc = bigram_model[unigram_doc]
    trigram_doc = trigram_model[bigram_doc]
    trigram_doc = [term for term in trigram_doc]
    
    #trigram_doc = ' '.join(trigram_doc)
    trigram_doc = [term for term in trigram_doc if len(term)>1]

    trigram_transformed_docs.append(trigram_doc)

    # create a bag-of-words representation
    doc_bow = ldaModel[1].doc2bow(trigram_doc)
    
    # create an LDA representation
    doc_lda = ldaModel[0][doc_bow]
    
    # sort with the most highly related topics first
    doc_lda = sorted(doc_lda, key=takeSecond, reverse=True)
    
    for topic_number, freq in doc_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        print('{:35} {}'.format(topic_names[topic_number],
                                round(freq, 3)))


Let's try some random documents and se what happens. Here again I will take documents with more than 150 words. 

In [57]:
test_docs=[text for text in test_documents if len(text.split())>150]

In [60]:
randInds=[459,672,1390,2000]
for i in randInds:
    doc=test_docs[i]
    #topic_id=test_docs[i][1]
    #print("Document from newsgroup: {} \n".format(real_topic_names[topic_id]))
    print(doc)
    print("")
    print("Topics from model:")
    (lda_description(doc))
    print("__________________________________________________")

i once heard an arguement from a xtian friend similar to this. christianity is a higher logic. athiest like u will not be able to understand it. your atheist logic is very low. only thru faith can we understand the higher logic in god. so i asked him, so what is this higher logic his answer, i dont know. this, the posting above highlights one of the worst things about xtainity. it is abundantly clear to both atheists xtains that their believe is both illogical irrational. their tactics, therefore to disregard logic rationality altogether. silly excuses such as the ones above and those such as, how can u trust science, science was invented by man, only goes to further show the weakness of their religion. in my country where xtainity was and still is rapidly growing, xtains never try to convert people by appealing to their brains or senses. they know it would be a fruitless act, given the irrational nature of their faith. they would wait until a person is in distress, then they would com

C:\Users\venciso\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\gensim\models\phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


religious/atheist discussion        0.437
christianity                        0.244
religion                            0.211
miscellaneous                       0.106
__________________________________________________
i have a mystery part labeled nec ac100. its from the low voltage supply of an nec multisync i monitor. its a three lead part in a square package like a volate regulator or power transistor. the board is labeled cr691 where the part goes. possibly an scr the pin labeled g on the board goes to a zener diode reference voltage the pin labeled t1 goes to the negative lead of a capacitor in the power supply, and the pin labeled t2 goes to the negative side of the bridge rectifier in the supply. if anyone can tell me what this is, or better yet, where i can buy one just like it, please email me at ck3iandrew.cmu.edu. ive called necs monitor repair number and not only do they not know what the part is, but they dont think that they can find one to sell to me... it makes no sens

In [62]:
randInds=[119,67,1860,2041]
for i in randInds:
    doc=test_docs[i]
    #topic_id=test_docs[i][1]
    #print("Document from newsgroup: {} \n".format(real_topic_names[topic_id]))
    print(doc)
    print("")
    print("Topics from model:")
    (lda_description(doc))
    print("__________________________________________________")

wrong about what i think they are correct in thinking that a wellplaced bomb or six would get headlines, but i think they are wrong if they think that you can set off bombs and still be a buddhist. maybe what we are seeing here is that chinese cultural genocide against the tibetans has worked well enough that some tibetans are now no longer buddhist and are instead willing to behave like the chinese occupiers. every action is its own reward. on the other hand, people who are aware of the occupation are mostly full of admiration for the peaceful way that tibetans have put up with it. and what does it cost us to admire them zip. yes they are, and whether this serves them well or not depends on whether they want buddhist principles or political independence. and without political independence can they preserve their cultural and religious traditions the chinese would certainly refer to them as terrorists, just as the hitler regime used to refer to european resistance movements as terroris

C:\Users\venciso\AppData\Local\Continuum\Anaconda2\envs\py36\lib\site-packages\gensim\models\phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


politics                            0.631
foreign politics                    0.142
medicine                            0.119
politics                            0.106
__________________________________________________
if i get a chance i will ask them this weekend. the words i have underlined are at the heart of the problem. a quick look doesnt do justice to the depth of the book of jeremiah. having studied the jeremiahezekial period solidly for over a year at one stage of my life, i have to say that there is a great deal of underlying theological meaning in the judgement prophesies. let me make one point. the clash between jeremiah and the false prophets was primarily in the theological realm. the false prophets understood their relatioship to god to be based on the covenant that the lord made with david. it is possible to trace within the pages of the old testament who this covenant, which was initially conditional on the continued obedience of davids descendants, came to be viewed 

In general topic prediction is performing well. In most cases the first or second topics seem to describe the theme of the document. 

**Summary and final notes**

In this notebook we have gone through all the necessary steps to create a model that infers topics from a given document.

One final note is that both the spacy and gensim modules support streaming and parallelisation which will be useful/necessary if the number of documents to analyse grows significantly. 

The code below just saves the necesary bits to make a class to use outside of the notebook. 

In [72]:
ldaModel[0].save('lda_final_model')

save_documents = open("bigram_model.pickle", "wb")
pickle.dump(bigram_model, save_documents)
save_documents.close()

save_documents = open("trigram_model.pickle", "wb")
pickle.dump(trigram_model, save_documents)
save_documents.close()

save_documents = open("trigram_dictionary.pickle", "wb")
pickle.dump(trigram_dictionary, save_documents)
save_documents.close()